In [2]:
import pandas as pd
import re
import numpy as np
import cPickle

In [3]:
df = pd.read_csv('data/mbu.csv', usecols=['Rating','Review'], sep=',')

In [4]:
df=df.dropna()

In [5]:
x=df.values

In [6]:
s=x[:,1]

In [7]:
sent=[]
vocab = {}

In [8]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),\.!?]", " ", string)
    #string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"e\.g\.,", " ", string)
    string = re.sub(r"a\.k\.a\.", " ", string)
    string = re.sub(r"i\.e\.,", " ", string)
    string = re.sub(r"i\.e\.", " ", string)
    #string = re.sub(r"\'ve", " \'ve", string)
    #string = re.sub(r"\'", "", string)
    #string = re.sub(r"\'re", " \'re", string)
    #string = re.sub(r"\'d", " \'d", string)
    #string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", "", string)
    #string = re.sub(r",", " , ", string)
    string = re.sub(r"br", "", string)
    string = re.sub(r"!", "", string)
    #string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    #string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", "", string)
    #string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", "", string)
    #string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\.", "", string)
    #string = re.sub(r"\.", " . ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"u\.s\.", " us ", string)
    return string.strip().lower()

In [9]:
for i in s:
    temp = clean_str(i).split()
    sent.append(temp)
    t = set(temp)
    for word in t:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

In [10]:
vocab

{'fawn': 3,
 'customerexcellent': 1,
 'nunnery': 1,
 'sowell': 1,
 'sonja': 17,
 'particularmike': 1,
 'woods': 72,
 'spiders': 1,
 'hanging': 151,
 'ultimatley': 1,
 'woody': 19,
 'comically': 1,
 'wantbill': 1,
 'dianostic': 2,
 'replacedthey': 1,
 'aldarwesh': 2,
 'glitters': 1,
 'slasorservice': 1,
 'mrbyon': 1,
 'sprague': 4,
 'brockton': 26,
 'gteborg': 1,
 'afantastic': 1,
 'igual': 1,
 'refunding': 16,
 'rickman': 7,
 'ioannidis': 5,
 'overcharge': 59,
 'financiamientogracias': 1,
 'mrgarvin': 1,
 'thanksmark': 1,
 'broward': 14,
 'bringing': 19,
 'expensewithout': 1,
 'purchasraz': 2,
 'hahahai': 1,
 'kaushal': 3,
 'grueling': 19,
 'wooden': 6,
 'biggggg': 1,
 'usebeginning': 1,
 'wednesday': 333,
 'nameindicates': 1,
 'advisethanks': 1,
 'whithers': 2,
 'servicebest': 4,
 'fuera': 3,
 'expeirence': 3,
 'luatich': 18,
 'onethanks': 2,
 'gaskets': 30,
 'convo': 10,
 'advisorsimply': 1,
 '270': 18,
 '271': 5,
 '272': 5,
 '274': 1,
 '275': 12,
 'gorman': 15,
 '277': 1,
 '278': 1,

In [11]:
ixtoword = {}

In [12]:
ixtoword[0] = 'END'
ixtoword[1] = 'UNK'
wordtoix = {}
wordtoix['END'] = 0
wordtoix['UNK'] = 1
ix = 2

In [13]:
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [14]:
ixtoword

{0: 'END',
 1: 'UNK',
 2: 'fawn',
 3: 'customerexcellent',
 4: 'nunnery',
 5: 'sowell',
 6: 'sonja',
 7: 'particularmike',
 8: 'woods',
 9: 'spiders',
 10: 'hanging',
 11: 'ultimatley',
 12: 'woody',
 13: 'comically',
 14: 'wantbill',
 15: 'dianostic',
 16: 'replacedthey',
 17: 'aldarwesh',
 18: 'glitters',
 19: 'slasorservice',
 20: 'mrbyon',
 21: 'sprague',
 22: 'brockton',
 23: 'gteborg',
 24: 'afantastic',
 25: 'igual',
 26: 'refunding',
 27: 'rickman',
 28: 'ioannidis',
 29: 'overcharge',
 30: 'financiamientogracias',
 31: 'mrgarvin',
 32: 'thanksmark',
 33: 'broward',
 34: 'bringing',
 35: 'expensewithout',
 36: 'purchasraz',
 37: 'hahahai',
 38: 'kaushal',
 39: 'grueling',
 40: 'wooden',
 41: 'biggggg',
 42: 'usebeginning',
 43: 'wednesday',
 44: 'nameindicates',
 45: 'advisethanks',
 46: 'whithers',
 47: 'servicebest',
 48: 'fuera',
 49: 'expeirence',
 50: 'luatich',
 51: 'onethanks',
 52: 'gaskets',
 53: 'convo',
 54: 'advisorsimply',
 55: '270',
 56: '271',
 57: '272',
 58: '

In [15]:
def convert_word_to_ix(data):
    result = []
    for sent in data:
        temp = []
        for w in sent:
            if w in wordtoix:
                temp.append(wordtoix[w])
            else:
                temp.append(1)
        temp.append(0)
        result.append(temp)
    return result

In [16]:
train_sent = convert_word_to_ix(sent)

In [17]:
train_sent[:10]

[[60155,
  25470,
  61593,
  56258,
  67176,
  19002,
  19200,
  15417,
  57492,
  12188,
  32662,
  45823,
  16774,
  39381,
  20319,
  23657,
  16766,
  69243,
  27079,
  2745,
  58037,
  66832,
  19200,
  78285,
  4524,
  69243,
  49499,
  44401,
  14939,
  43171,
  57726,
  21417,
  58037,
  46189,
  14939,
  33369,
  38366,
  7379,
  60155,
  69243,
  10249,
  11187,
  43551,
  38025,
  52819,
  60155,
  17765,
  55419,
  19200,
  13995,
  46157,
  15417,
  25304,
  60155,
  43769,
  36385,
  32662,
  55419,
  4809,
  42486,
  44268,
  6528,
  10249,
  10158,
  63753,
  0],
 [14939, 38495, 0],
 [44671,
  13042,
  19211,
  46072,
  54891,
  18305,
  29294,
  50144,
  50909,
  44401,
  61508,
  81626,
  74735,
  67214,
  63584,
  20321,
  67115,
  44401,
  14939,
  82492,
  46072,
  61879,
  58037,
  37707,
  0],
 [45453,
  69243,
  20316,
  60155,
  21417,
  12183,
  79945,
  81031,
  57726,
  39078,
  47599,
  12355,
  0],
 [23269, 39903, 26194, 78285, 69340, 37707, 34991, 31133, 

In [18]:
train_y=x[:,0]

In [19]:
train_l=[int(round(y)) for y in train_y]

In [20]:
train_l[:10]

[5, 4, 5, 3, 4, 1, 5, 5, 5, 5]

In [21]:
train_lab=np.zeros((len(train_l),5))

In [22]:
for i,j in zip(range(len(train_lab)),train_l):
    train_lab[i][j-1]=1 

In [23]:
train_lab[:10]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [24]:
len(train_sent)

193777

In [25]:
len(train_lab)

193777

In [26]:
l=len(train_lab)

In [27]:
s1=int(round(len(train_lab)*0.8))

In [28]:
s1

155022

In [29]:
s2=int(round(len(train_lab)*0.9))

In [30]:
s2

174399

In [31]:
train_x = train_sent[:s1]
train_y = train_lab[:s1]
val_x = train_sent[s1:s2]
val_y = train_lab[s1:s2]
test_x = train_sent[s2:]
test_y = train_lab[s2:]

In [32]:
cPickle.dump([train_x, val_x, test_x, train_y, val_y, test_y, wordtoix, ixtoword], open("mbu0.p", "wb"))

In [148]:
len(vocab)

62296

In [149]:
train_y[:10]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])